In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from skfeature.function.similarity_based import fisher_score
import warnings
# warnings.filterwarnings('ignore')
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)

# EDA

In [20]:
df = pd.read_csv('OnlineNewsPopularity.csv')
df.head()
#Check Nullity
df.isnull().sum() #It appears that the data has no null values
#For ' n_tokens_content', if it is 0, it means that the artitle is empty, therefore, let's remove those rows
print(df.shape)
df = df[df[' n_tokens_content'] != 0]
print(df.shape)
#Drop columns that are obbiously not going to be useful
df.drop(['url', ' timedelta', ' n_non_stop_words', ' n_non_stop_unique_tokens'], axis=1, inplace=True)

(39644, 61)
(38463, 61)


In [21]:
df.describe()

,n_tokens_title,n_tokens_content,n_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
count,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,...,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000,38463.000000
mean,10.382419,563.295375,0.565049,11.217872,3.394769,4.563061,1.263786,4.687892,7.215012,0.054000,...,0.098376,0.779963,-0.267493,-0.537970,-0.110801,0.280573,0.070997,0.342431,0.154930,3355.360398
std,2.113800,468.299538,3.573022,11.340580,3.869773,8.295365,4.164896,0.283231,1.916459,0.226021,...,0.070382,0.212509,0.121174,0.279703,0.094919,0.323561,0.264338,0.188606,0.225636,11585.968776
min,2.000000,18.000000,0.114964,0.000000,0.000000,0.000000,0.000000,3.600000,1.000000,0.000000,...,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,1.000000
25%,9.000000,259.000000,0.477419,5.000000,1.000000,1.000000,0.000000,4.496250,6.000000,0.000000,...,0.050000,0.600000,-0.331532,-0.714286,-0.125000,0.000000,0.000000,0.166667,0.000000,945.000000
50%,10.000000,423.000000,0.542986,8.000000,3.000000,1.000000,0.000000,4.674121,7.000000,0.000000,...,0.100000,0.800000,-0.257738,-0.500000,-0.100000,0.125000,0.000000,0.500000,0.000000,1400.000000
75%,12.000000,729.000000,0.611111,14.000000,4.000000,4.000000,1.000000,4.861901,9.000000,0.000000,...,0.100000,1.000000,-0.193415,-0.312500,-0.050000,0.500000,0.136364,0.500000,0.250000,2700.000000
max,23.000000,8474.000000,701.000000,304.000000,116.000000,128.000000,91.000000,8.041534,10.000000,1.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.500000,1.000000,843300.000000


In [14]:
#25 Percentile
print(df[' shares'].quantile(q=0.25))

#50 Percentile
print(df[' shares'].quantile(q=0.50))

#75 Percentile
print(df[' shares'].quantile(q=0.75))


945.0
1400.0
2700.0


In [22]:
#Convert into binary classfication problem
#Use median to create label: 0:bad 1:good
print(df[' shares'].median())
df[' popularity'] = df[' shares'].apply(lambda x: 0 if x <1400 else 1)
df.head()

1400.0


,n_tokens_title,n_tokens_content,n_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,popularity
0,12.0,219.0,0.663594,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0
1,9.0,255.0,0.604743,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0
2,9.0,211.0,0.575130,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,1
3,9.0,531.0,0.503788,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,0
4,13.0,1072.0,0.415646,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0


In [23]:
#Some of the features are related to each other such as ' weekday_is_monday' and ' weekday_is_tuesday' and ' data_channel_is_lifestyle' and ' data_channel_is_entertainment'. This violates non-multicollinearity of model such as linear regression
#Merging 
dayMerge = df[[' weekday_is_monday',' weekday_is_tuesday',' weekday_is_wednesday',' weekday_is_thursday',' weekday_is_friday',' weekday_is_saturday',' weekday_is_sunday', ]]
day_arr = []
for r in list(range(dayMerge.shape[0])):
    for c in list(range(dayMerge.shape[1])):
        if ((c==0) and (dayMerge.iloc[r,c])==1):
            day_arr.append(0)
        elif ((c==1) and (dayMerge.iloc[r,c])==1):
            day_arr.append(1)
        elif ((c==2) and (dayMerge.iloc[r,c])==1):
            day_arr.append(2)
        elif ((c==3) and (dayMerge.iloc[r,c])==1):
            day_arr.append(3)
        elif ((c==4) and (dayMerge.iloc[r,c])==1):
            day_arr.append(4)
        elif ((c==5) and (dayMerge.iloc[r,c])==1):
            day_arr.append(5) 
        elif ((c==6) and (dayMerge.iloc[r,c])==1):
            day_arr.append(6)
            
channelMerge=df[[' data_channel_is_lifestyle',' data_channel_is_entertainment' ,' data_channel_is_bus',
                        ' data_channel_is_socmed' ,' data_channel_is_tech',' data_channel_is_world' ]]

channel_arr = []

for r in list(range(channelMerge.shape[0])):
    if (((channelMerge.iloc[r,0])==0) and ((channelMerge.iloc[r,1])==0) and ((channelMerge.iloc[r,2])==0) and ((channelMerge.iloc[r,3])==0) and ((channelMerge.iloc[r,4])==0) and ((channelMerge.iloc[r,5])==0)):
        channel_arr.append(6)
    for c in list(range(channelMerge.shape[1])):
        if ((c==0) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(0)
        elif ((c==1) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(1)
        elif ((c==2) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(2)
        elif ((c==3) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(3)
        elif ((c==4) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(4)
        elif ((c==5) and (channelMerge.iloc[r,c])==1):
            channel_arr.append(5)
            
df.insert(loc=11, column='weekdays', value=day_arr)
df.insert(loc=12, column='data_channel', value=channel_arr)

df.drop(labels=[' data_channel_is_lifestyle',' data_channel_is_entertainment' ,' data_channel_is_bus',
                  ' data_channel_is_socmed' ,' data_channel_is_tech',' data_channel_is_world', 
                  ' weekday_is_monday',' weekday_is_tuesday',' weekday_is_wednesday', 
                  ' weekday_is_thursday', ' weekday_is_friday',' weekday_is_saturday' ,' weekday_is_sunday', ' shares'], axis = 1, inplace=True)
df.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,weekdays,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,popularity
0,12.0,219.0,0.663594,4.0,2.0,1.0,0.0,4.680365,5.0,0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,0
1,9.0,255.0,0.604743,3.0,1.0,1.0,0.0,4.913725,4.0,0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,0
2,9.0,211.0,0.575130,3.0,1.0,1.0,0.0,4.393365,6.0,0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1
3,9.0,531.0,0.503788,9.0,0.0,1.0,0.0,4.404896,7.0,0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,0
4,13.0,1072.0,0.415646,19.0,19.0,20.0,0.0,4.682836,7.0,0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,0


In [24]:
#Check Class Balance
class_counts = df.groupby(' popularity').size().reset_index()
class_counts.columns = ['Popularity','No of articles']
print(class_counts)

weekdays_data = df.groupby('weekdays').size().reset_index()
weekdays_data.columns = ['weekdays','count']
print(weekdays_data)

channel_data = df.groupby('data_channel').size().reset_index()
channel_data.columns = ['data_channel','count']
print(channel_data)

   Popularity  No of articles
0           0           17999
1           1           20464
   weekdays  count
0         0   6471
1         1   7171
2         2   7205
3         3   7052
4         4   5538
5         5   2369
6         6   2657
   data_channel  count
0             0   2077
1             1   6856
2             2   6235
3             3   2311
4             4   7325
5             5   8168
6             6   5491


# Feature Selection

## L1 Feature Selection

In [ ]:
#Attempting Lasso(L1) feature selection
# shares data is not needed for classification
X = df.drop(labels=['popularity'], axis = 1, inplace=False)
y = df['popularity']
X= pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False)



# Standardize the train and test sample
scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)

# Perform GridSearchCV to tune best-fit LR model
param = {'C': [10**-2,10**-1,10**0,10**1,10**2]}

lr_model = LogisticRegression(penalty='l1', solver='liblinear')
lr_model.fit(X_train, y_train)


print("Redundant Feature Count: ", sum(lr_model.coef_[0]==0))
print("Redundant Feature Names: ", list(pd.Series(X_train.columns)[list(lr_model.coef_[0]==0)]))

In [ ]:
df.drop(labels=[' global_rate_negative_words', ' avg_positive_polarity', ' global_sentiment_polarity', ' avg_negative_polarity', ' max_negative_polarity'], axis = 1, inplace=True)

## Fisher Score Feature Selection

In [26]:
# scaler = StandardScaler()
# X = df.drop(labels=[' popularity'], axis = 1, inplace=False)
# y = df[' popularity']
# X = pd.get_dummies(X)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False)
# X_std = scaler.fit_transform(X_train)
# from skfeature.function.similarity_based import fisher_score
# ranks = fisher_score.fisher_score(X_std, y_train)
# print(ranks)

/opt/anaconda3/lib/python3.8/site-packages/skfeature/utility/construct_W.py:194: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  class_idx_all = class_idx[:, np.newaxis] & class_idx[np.newaxis, :]
/opt/anaconda3/lib/python3.8/site-packages/skfeature/utility/construct_W.py:194: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  class_idx_all = class_idx[:, np.newaxis] & class_idx[np.newaxis, :]


KeyboardInterrupt: 

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# feat_importance = pd.Series(ranks, X.columns[0: len(X.columns)])
# feat_importance.plot(kind='barh', color = 'teal')
# fig = plt.figure() 
# fig.set_size_inches(50, 50)
# plt.show()

In [ ]:
# idx_lst = []
# score_map = []
# for idx, val in enumerate(ranks):
#     if val < 5:
#         idx_lst.append(idx)
#         score_map.append((idx, val))
# print(score_map)
# print("Redundant Feature Names: ", list(pd.Series(X_train.columns)[idx_lst]))

In [27]:
# df.drop(labels=list(pd.Series(X_train.columns)[idx_lst]), axis = 1, inplace=True)
df.drop(labels=[' global_rate_negative_words', ' avg_positive_polarity', ' global_sentiment_polarity', ' avg_negative_polarity', ' max_negative_polarity'], axis = 1, inplace=True)

# Model Training

In [28]:
from sklearn.preprocessing import LabelEncoder
X_std = scaler.fit_transform(df)
df = pd.DataFrame(X_std, columns = df.columns[0: len(df.columns)])
X = df.drop(labels=[' popularity'], axis = 1, inplace=False)
y = df[' popularity']
labelEn = LabelEncoder()
y = labelEn.fit_transform(y)
# class_names = labelEn.classes_

X= pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle = False)


## Support Vector Machine

In [ ]:
# svm = SVC(gamma='auto')
# svm.fit(X_train, y_train)

## Logistic Regression

In [ ]:
# logistic_regression = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=7000)
# logistic_regression.fit(X_train, y_train)


## Tree-based Method: Random Forest

In [29]:
rf_classfication = RandomForestClassifier(n_estimators=1000, n_jobs=-1, max_depth=50,random_state=666)
rf_classfication.fit(X_train, y_train)

RandomForestClassifier(max_depth=50, n_estimators=1000, n_jobs=-1,
                       random_state=666)

## Tree-based Method: XG Boost

In [30]:
from xgboost import XGBClassifier
xgboost = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=6, monotone_constraints='()',
       n_estimators=400, n_jobs=8, num_parallel_tree=1, random_state=0,
            reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

xgboost.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=6,
              missing=nan, monotone_constraints='()', n_estimators=400,
              n_jobs=8, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

## Deep Learning

In [ ]:
# import torch as T
# class Net(T.nn.Module):
#   def __init__(self):
#     super(Net, self).__init__()
#     self.hid1 = T.nn.Linear(8, 10)  # 8-(10-10)-1
#     self.hid2 = T.nn.Linear(10, 10)
#     self.oupt = T.nn.Linear(10, 1)

#     T.nn.init.xavier_uniform_(self.hid1.weight)
#     T.nn.init.zeros_(self.hid1.bias)
#     T.nn.init.xavier_uniform_(self.hid2.weight)
#     T.nn.init.zeros_(self.hid2.bias)
#     T.nn.init.xavier_uniform_(self.oupt.weight)
#     T.nn.init.zeros_(self.oupt.bias)

#   def forward(self, x):
#     z = T.relu(self.hid1(x))
#     z = T.relu(self.hid2(z))
#     z = self.oupt(z)  # no activation
#     return z

# bat_size = 10
# net = Net()

# Evaluation

In [ ]:
#Evaluateing Logistic Regression
# y_pred = logistic_regression.predict(X_test)
# print("F1 Score: ",f1_score(y_test, y_pred, average='macro'))
#print("Accuracy: ",accuracy_score(y_test, y_pred))
#F1 Score:  0.31860939565338836

In [31]:
#Evaluating RF
y_pred = rf_classfication.predict(X_test)
print("F1 Score: ",f1_score(y_test, y_pred, average='macro'))
print("Accuracy: ",accuracy_score(y_test, y_pred))

F1 Score:  0.6819314882506857
Accuracy:  0.6820348383742092


In [ ]:
#Evaluating SVM
# y_pred = svm.predict(X_test)
# print("F1 Score: ",f1_score(y_test, y_pred, average='macro'))
#print("Accuracy: ",accuracy_score(y_test, y_pred))
#F1 Score:  0.10067998398348055

In [32]:
#Evaluating XG Boost
y_pred = xgboost.predict(X_test)
print("F1 Score: ",f1_score(y_test, y_pred, average='macro'))
print("Accuracy: ",accuracy_score(y_test, y_pred))

F1 Score:  0.6763074207537915
Accuracy:  0.6764017679174972


# Conclusions and Findings